# Run MPNN interface design on the bound states

### Imports

In [1]:
%load_ext lab_black
# Python standard library
from glob import glob
import os
import socket
import sys

# 3rd party library imports
import dask
import matplotlib.pyplot as plt
import pandas as pd
import pyrosetta
import numpy as np
import scipy
import seaborn as sns
from tqdm.auto import tqdm  # jupyter compatible progress bar

tqdm.pandas()  # link tqdm to pandas
# Notebook magic
# save plots in the notebook
%matplotlib inline
# reloads modules automatically before executing cells
%load_ext autoreload
%autoreload 2
print(f"running in directory: {os.getcwd()}")  # where are we?
print(f"running on node: {socket.gethostname()}")  # what node are we on?

running in directory: /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties
running on node: dig130


### Set working directory to the root of the crispy_shifty repo
TODO set to projects dir

In [2]:
os.chdir("/home/pleung/projects/crispy_shifty")
# os.chdir("/projects/crispy_shifty")

### Run MPNN on the interfaces
TODO

In [3]:
# from crispy_shifty.utils.io import gen_array_tasks

# simulation_name = "02_mpnn_bound_states"
# design_list_file = os.path.join(
#     os.getcwd(), "projects/crispy_shifties/02_loop_bound_states/looped_states.list"
# )
# output_path = os.path.join(os.getcwd(), f"projects/crispy_shifties/{simulation_name}")
# options = " ".join(
#     [
#         "out:level 200",
# #         "corrections::beta_nov16 true",
#     ]
# )

# gen_array_tasks(
#     distribute_func="crispy_shifty.protocols.mpnn.mpnn_bound_state",
#     design_list_file=design_list_file,
#     output_path=output_path,
#     queue="medium",
#     memory="4G",
#     nstruct=1,
#     nstruct_per_task=1,
#     options=options,
#     simulation_name=simulation_name,
# )

In [4]:
# !sbatch -a 1-$(cat /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/01_loop_bound_states/tasks.cmds | wc -l) /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/01_loop_bound_states/run.sh

### Collect scorefiles of designed bound states and concatenate
TODO change to projects dir

In [5]:
# sys.path.insert(0, "~/projects/crispy_shifty")  # TODO
# from crispy_shifty.utils.io import collect_score_file

# simulation_name = "02_mpnn_bound_states"
# output_path = os.path.join(os.getcwd(), f"projects/crispy_shifties/{simulation_name}")

# if not os.path.exists(os.path.join(output_path, "scores.json")):
#     collect_score_file(output_path, "scores")

### Load resulting concatenated scorefile
TODO change to projects dir

In [6]:
# sys.path.insert(0, "~/projects/crispy_shifty")  # TODO
# from crispy_shifty.utils.io import parse_scorefile_linear

# output_path = os.path.join(os.getcwd(), f"projects/crispy_shifties/{simulation_name}")

# scores_df = parse_scorefile_linear(os.path.join(output_path, "scores.json"))
# scores_df = scores_df.convert_dtypes()

### Setup for plotting

In [7]:
# sns.set(
#     context="talk",
#     font_scale=1,  # make the font larger; default is pretty small
#     style="ticks",  # make the background white with black lines
#     palette="colorblind",  # a color palette that is colorblind friendly!
# )

### Data exploration
Gonna remove the Rosetta sfxn scoreterms for now

In [8]:
# from crispy_shifty.protocols.design import beta_nov16_terms

# scores_df = scores_df[
#     [term for term in scores_df.columns if term not in beta_nov16_terms]
# ]
# print(len(scores_df))
# scores_df.columns

### Save a list of outputs

In [9]:
# simulation_name = "02_mpnn_bound_states"
# output_path = os.path.join(os.getcwd(), f"projects/crispy_shifties/{simulation_name}")

# with open(os.path.join(output_path, "mpnn_states.list"), "w") as f:
#     for path in tqdm(scores_df.index):
#         print(path, file=f)

### Prototyping blocks

test `mpnn_bound_state`

In [10]:
%%time 
import pyrosetta

pyrosetta.init()


sys.path.insert(0, "~/projects/crispy_shifty/") # TODO projects
from crispy_shifty.protocols.mpnn import mpnn_bound_state

t = mpnn_bound_state(
        None,
        **{
            'pdb_path': '/mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/01_loop_bound_states/decoys/0001/01_loop_bound_states_17f57e75865441a78a0057fb8081b4de.pdb.bz2',
        }
)

PyRosetta-4 2021 [Rosetta PyRosetta4.conda.linux.cxx11thread.serialization.CentOS.python38.Release 2021.50+release.4ff291ed8257ec2d0cd4d96ae4289e1d39ce1007 2021-12-16T00:25:15] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.conda.linux.cxx11thread.serialization.CentOS.python38.Release r306 2021.50+release.4ff291e 4ff291ed8257ec2d0cd4d96ae4289e1d39ce1007 http://www.pyrosetta.org 2021-12-16T00:25:15
core.init: command: PyRosetta -ex1 -ex2aro -database /projects/crispy_shifty/envs/crispy/lib/python3.8/site-packages/pyrosetta/database
basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=2046730043 seed_offset=0 real_seed=2046730043 thread_index=0
basic.random.init_random_generator: RandomGenerator:init: Normal mode, seed=2046730043 RG_type=mt19937
CPU times: 

In [11]:
for i, tppose in enumerate(t):
    tppose.pose.dump_pdb(f"{i}.pdb")

/projects/crispy_shifty/envs/crispy/lib/python3.8/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


https://docs.anaconda.com/anaconda/install

core.chemical.GlobalResidueTypeSet: Finished initializing fa_standard residue type set.  Created 983 residue types
core.chemical.GlobalResidueTypeSet: Total time to initialize 1.21504 seconds.
0.05 min: Setting up design selector
0.05 min: Designing interface with MPNN
/bin/sh: /projects/crispy_shifty/mpnn/parse_multiple_chains.py: No such file or directory

/projects/crispy_shifty/mpnn/mpnn_run.py --backbone_noise 0.05 --checkpoint_path /projects/ml/struc2seq/data_for_complexes/training_scripts/paper_experiments/model_outputs/p10/checkpoints/epoch51_step255000.pt --decoding_order random --hidden_dim 192 --max_length 10000 --num_connections 64 --num_layers 3 --protein_features full --batch_size 8 --num_seq_per_target 64 --omit_AAs X --sampling_temp 0.1 --jsonl_path /scratch/pleung/4440023/696e0e67b4f146eb8e5411617e4de04e/biounits.jsonl --chain_id_jsonl /scratch/pleung/4440023/696e0e67b4f146eb8e5411617e4de04e/chain_id.jsonl --fixed_positions_j

FileNotFoundError: [Errno 2] No such file or directory: '/scratch/pleung/4440023/696e0e67b4f146eb8e5411617e4de04e/alignments/tmp.fa'

In [ ]:
import pyrosetta.distributed.viewer as viewer

ppose = pyrosetta.distributed.io.pose_from_file("test.pdb")
view = viewer.init(ppose, window_size=(1600, 1200))
view.add(viewer.setStyle())
view.add(viewer.setStyle(colorscheme="whiteCarbon", radius=0.10))
view.add(viewer.setHydrogenBonds())
view.add(viewer.setHydrogens(polar_only=True))
view.add(viewer.setDisulfides(radius=0.25))
view()